In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

import os
hugging_face_token = os.environ.get("HF_TOKEN")

In [ ]:
!pip install -q gdown pypdf transformers bitsandbytes
!pip install -q accelerate
!pip install -q langchain
!pip install -q sentence-transformers
!pip install -q cohere llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.7 MB/s eta 0:0

In [ ]:
import os
import torch
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import bitsandbytes as bnb

In [ ]:
import zipfile
!gdown 1YaSZyZfJFDQTbUaVfp0oPloF6ntjy5S6
!gdown 1xzP0D_aRrMl-Ly7qR_O6Bq-lsrKCYjQH
with zipfile.ZipFile("./dental-index-finetuned.zip", 'r') as zip_ref: zip_ref.extractall("./")
with zipfile.ZipFile("./finetuned_embedding_model.zip", 'r') as zip_ref: zip_ref.extractall("./")

Downloading...
From: https://drive.google.com/uc?id=1YaSZyZfJFDQTbUaVfp0oPloF6ntjy5S6
To: /content/finetuned_embedding_model.zip
100% 2.19M/2.19M [00:00<00:00, 203MB/s]
Downloading...
From: https://drive.google.com/uc?id=1xzP0D_aRrMl-Ly7qR_O6Bq-lsrKCYjQH
To: /content/dental-index-finetuned.zip
100% 103M/103M [00:03<00:00, 26.9MB/s] 


In [ ]:

hugging_face_model_id = "faisalahmedsifat/llama2-7b-chat-hf-base-pretrained"
cache_dir = "./model/"

# Fine tuned Embedding settings
index_persist_finetuned_dir='./index-finetuned'
useFinetunedEmbedding=False

if useFinetunedEmbedding == False:
  !mkdir "index"
  index_persist_dir = "./index"
else:
  index_persist_dir = index_persist_finetuned_dir

environment = "COLAB" # either COLAB or KAGGLE

if environment == "KAGGLE":
  dental_corpus_zip_path = "/kaggle/working/dental-guideline-dataset.zip"
  index_zip_path = "/kaggle/working/dental-guidelines-index.zip"
  embedding_finetuning_data = '/kaggle/working/embedding-finetuning-dataset.zip'
else:
  dental_corpus_zip_path = "./dental-guideline-dataset.zip"
  index_zip_path = "./dental-guidelines-index.zip"
  embedding_finetuning_data = './embedding-finetuning-dataset.zip'

In [ ]:
login(token=hugging_face_token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import zipfile

""" Only one of the following is needed """
""" If the Dataset Zip file is downloaded then the indexes will be build (again!) """
""" If the Dataset Zip file is not downloaded then make sure you download the indexes """

# Download Dataset Zip and unzip
# !gdown 1izm1q-HXwgPhNICZmKLnis7JkU9Co79L
# with zipfile.ZipFile(dental_corpus_zip_path, 'r') as zip_ref: zip_ref.extractall("./")

# Download Nonfinetuned Index Zip and unzip
if useFinetunedEmbedding == False:
  !gdown 1l3aFkVeW9hn2d0Mb9QFVxTy0hrZd8BNG
  with zipfile.ZipFile(index_zip_path, 'r') as zip_ref: zip_ref.extractall("./")

# Download embedding finetuning data
# !gdown 1AuEqYcI0JRVZpPvYL0XtzejZRMMgTwsN
# with zipfile.ZipFile(embedding_finetuning_data, 'r') as zip_ref: zip_ref.extractall("./")

Downloading...
From: https://drive.google.com/uc?id=1l3aFkVeW9hn2d0Mb9QFVxTy0hrZd8BNG
To: /content/dental-guidelines-index.zip
100% 103M/103M [00:03<00:00, 33.7MB/s] 


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", cache_dir=cache_dir, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(hugging_face_model_id, cache_dir=cache_dir, device_map="auto", quantization_config=bnb_config)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

(…)at-hf/resolve/main/tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)2-7b-chat-hf/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)-hf/resolve/main/special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

(…)base-pretrained/resolve/main/config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

(…)ined/resolve/main/generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
def prompt(prompt, model, streamer):
  model_inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
  model_output = model.generate(**model_inputs, streamer=streamer, use_cache=True, max_new_tokens=float('inf'))
  model_output_text = tokenizer.decode(model_output[0][model_inputs.input_ids.shape[-1]+1:], skip_special_tokens=True)
  return model_output_text

In [ ]:
# Import the prompt wrapper...but for llama index
from llama_index.prompts.prompts import SimpleInputPrompt
# Create a system prompt
system_prompt = """<<SYS>>
    You will be asked a question and you have to give a answer of the question directly and to the point answer. The domain is dentistry related. Answer within 50 words short.
<</SYS>>
"""

# Import the llama index HF Wrapper
from llama_index.llms import HuggingFaceLLM

# Create a HF LLM using the llama index wrapper
llm = HuggingFaceLLM(context_window=4096,
                    max_new_tokens=60,
                    system_prompt=system_prompt,
                    model=model,
                    tokenizer=tokenizer)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Bring in embeddings wrapper
from llama_index.embeddings import LangchainEmbedding
# Bring in HF embeddings - need these to represent document chunks
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# Create and dl embeddings instance
embeddings=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")
)

if useFinetunedEmbedding == True:
  from llama_index.embeddings import LinearAdapterEmbeddingModel
  embed_model = LinearAdapterEmbeddingModel(embeddings, "./finetuned_embedding_model")

(…)851d5dd1af673670cdb299753/.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

(…)1af673670cdb299753/1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(…)6e48e851d5dd1af673670cdb299753/README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

(…)48e851d5dd1af673670cdb299753/config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

(…)299753/config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

(…)851d5dd1af673670cdb299753/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)1af673670cdb299753/tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

(…)51d5dd1af673670cdb299753/train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

(…)6e48e851d5dd1af673670cdb299753/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)8e851d5dd1af673670cdb299753/modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
from llama_index import ServiceContext, VectorStoreIndex
from llama_index.schema import TextNode
from tqdm.notebook import tqdm
import pandas as pd

In [ ]:
# # eval_utils.py
# ###############################
# from llama_index.schema import TextNode
# from llama_index import ServiceContext, VectorStoreIndex
# import pandas as pd
# from tqdm import tqdm


# def evaluate(
#     dataset,
#     embed_model,
#     llm,
#     top_k=5,
#     verbose=False,
# ):
#     corpus = dataset.corpus
#     queries = dataset.queries
#     relevant_docs = dataset.relevant_docs

#     service_context = ServiceContext.from_defaults(chunk_size=1024,llm=llm,embed_model=embed_model)
#     nodes = [TextNode(id_=id_, text=text) for id_, text in corpus.items()]
#     index = VectorStoreIndex(
#         nodes, service_context=service_context, show_progress=True
#     )
#     retriever = index.as_retriever(similarity_top_k=top_k)

#     eval_results = []
#     for query_id, query in tqdm(queries.items()):
#         retrieved_nodes = retriever.retrieve(query)
#         retrieved_ids = [node.node.node_id for node in retrieved_nodes]
#         expected_id = relevant_docs[query_id][0]

#         rank = None
#         for idx, id in enumerate(retrieved_ids):
#             if id == expected_id:
#                 rank = idx + 1
#                 break

#         is_hit = rank is not None  # assume 1 relevant doc
#         mrr = 0 if rank is None else 1 / rank

#         eval_result = {
#             "is_hit": is_hit,
#             "mrr": mrr,
#             "retrieved": retrieved_ids,
#             "expected": expected_id,
#             "query": query_id,
#         }
#         eval_results.append(eval_result)
#     return eval_results


# def display_results(names, results_arr):
#     """Display results from evaluate."""

#     hit_rates = []
#     mrrs = []
#     for name, results in zip(names, results_arr):
#         results_df = pd.DataFrame(results)
#         hit_rate = results_df["is_hit"].mean()
#         mrr = results_df["mrr"].mean()
#         hit_rates.append(hit_rate)
#         mrrs.append(mrr)

#     final_df = pd.DataFrame(
#         {"retrievers": names, "hit_rate": hit_rates, "mrr": mrrs}
#     )
#     display(final_df)

In [ ]:
from llama_index.finetuning import EmbeddingAdapterFinetuneEngine, EmbeddingQAFinetuneDataset
from sentence_transformers import SentenceTransformer

In [ ]:
# train_dataset = EmbeddingQAFinetuneDataset.from_json("/content/EmbeddingFinetuningData/train_corpus.json")
# val_dataset = EmbeddingQAFinetuneDataset.from_json("/content/EmbeddingFinetuningData/val_dataset.json")

In [ ]:
# finetune_engine = EmbeddingAdapterFinetuneEngine(
#     train_dataset,
#     embeddings,
#     model_output_path="./model",
#     bias=True,
#     epochs=10,
#     verbose=True,
#     optimizer_class=torch.optim.Adam,
#     optimizer_params={"lr": 0.001}
# )
# finetune_engine.finetune()

# embeddings_nonfinetuned=LangchainEmbedding(
#     HuggingFaceEmbeddings(model_name="all-mpnet-base-v2") # Change best model from https://www.sbert.net/docs/pretrained_models.html
# )

# mpnet_finetuned_results = evaluate(val_dataset, embeddings, llm)
# mpnet_nonfinetuned_results = evaluate(val_dataset, embeddings_nonfinetuned, llm)
# display_results(
#     ["finetuned", "non-finetuned"], [mpnet_finetuned_results, mpnet_nonfinetuned_results]
# )

In [ ]:
# Bring in stuff to change service context
from llama_index import set_global_service_context
from llama_index import ServiceContext

# Create new service context instance
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embeddings
)
# And set the service context
set_global_service_context(service_context)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
index_persist_dir

'./index'

In [ ]:
from llama_index import SimpleDirectoryReader, VectorStoreIndex, StorageContext, load_index_from_storage

# if not os.path.exists("./index"):
#     # load the documents and create the index
#     documents = SimpleDirectoryReader("dataset").load_data()
#     index = VectorStoreIndex.from_documents(documents, service_context=service_context)
#     # store it for later
#     index.storage_context.persist(persist_dir=index_persist_dir)
# else:
#load the existing index
storage_context = StorageContext.from_defaults(persist_dir=index_persist_dir)
index = load_index_from_storage(storage_context)


In [ ]:
def prompt_formatting(query_str):
    system_prompt = """<<SYS>>
    You will be asked a question and you have to give a answer of the question directly and to the point answer. The domain is dentistry related. Answer within 50 words short.
    <</SYS>>
    """

    instruction_prompt = f"[INST] Answer the following question and give to the point answer: {query_str} [/INST]"
    return system_prompt + instruction_prompt

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=1)

In [ ]:
def ask(prompt, engine):
    torch.cuda.empty_cache()
    current_prompt = prompt_formatting(prompt)
    return engine.query(current_prompt).response

# Evaluating Test Set

In [ ]:
!gdown 1dbZV5N6y9XIyHgSmLbasgFmD0Cqd1h_P

Downloading...
From: https://drive.google.com/uc?id=1dbZV5N6y9XIyHgSmLbasgFmD0Cqd1h_P
To: /content/evaluation-questions.jsonl
100% 5.92k/5.92k [00:00<00:00, 23.9MB/s]


In [ ]:
input_json_data = '''
{"0": {"Source": "1- OPERATIVE BASICS.pdf’ ‘Page 16", "Question": "What types of supplementary pins have been recommended in the past to enhance retention, and what is the recommended depth for the pin hole into dentine?", "Answer": "Cemented, friction grip, and self-threading pins have been recommended in the past to enhance retention. The pin hole should be 2-3 mm deep into dentine."},
"1": {"Source": "2- DENTAL CARIES_FLUORIDES.pdf ‘ ‘Page 8", "Question": "What are the changes in risk profiles mentioned for Australia due to fluoridation?", "Answer": "The changes in risk profiles in Australia include:15-30 year olds and the elderly being at the greatest risk of active caries.A greater proportion of the population remaining caries-free, but a small proportion experiencing caries rates similar to those before fluoride supplementation.Caries being more easily hidden within dentine with little visual clinical evidence in enamel.The need for more thorough clinical examinations of both crowns and exposed roots to identify caries lesions at their earliest stage.Erosion caused by frequent use of dietary erosive acids and endogenous acids becoming a more frequent cause of tooth damage."},
"2": {"Source": "‘3- RESTORATIVE MATERIALS.pdf ‘ ‘Page 7’.", "Question": "Why is cast gold a good choice for tooth restoration?", "Answer": "Cast gold may be a good choice in some circumstances because it can be placed in thin section to protect and reinforce remaining tooth structure."},
"3": {"Source": "‘4- TOOTHWEAR_FRACTURES.pdf ‘ ‘Page 21’.", "Question": "What are the categories of longitudinal fractures and what is the severity order of them?", "Answer": "There are five categories of longitudinal fractures. From least to most severe they are (1) craze lines, (2) fractured cusp, (3) cracked tooth, (4) split tooth, and (5) vertical root fracture."},
"4": {"Source": "‘5-BLEACHING_DISCOLORATIONS ‘ ‘Page 7’.", "Question": "What bleaching chemicals are used?", "Answer": "The following names are the names of bleaching chemicals:Hydrogen PeroxideSodium perborateCarbamide peroxideSodium peroxyborate monohydrate (Amosan) (however, less used in today's world)Sodium hypochlorite"},
"5": {"Source": "‘1- ENDODONTICS BASICS.pdf ‘ ‘Page 19’.", "Question": "As per the task force of the American Heart Association with input from the American Dental Association recommendation guidelines for antibiotic prophylaxis, what is the regimen for amoxicillin in adults and children? And what is the dosage for allergic patients?", "Answer": "The regimen is:2 grams of amoxicillin 30 to 60 minutes preoperatively for adults.Children’s dosage is 50 mg/kg.For allergic patients, a good choice is clindamycin 600 mg 30 to 60 minutes before the procedure."},
"6": {"Source": "‘2- RESORPTION ‘ ‘Page 2’.", "Question": "What are the analogous terminologies of External surface resorption?", "Answer": "Surface resorption, Transient inflammatory resorption, and Cemental healing"},
"7": {"Source": "‘3- QUESTIONS.pdf’ ‘Page 2’", "Question": "Approximately how many odontoblasts are present in the crown of a newly erupted tooth?", "Answer": "Between 45,000 and 65,000/mm2 in that area."},
"8": {"Source": "‘5- QUICK REVISION.pdf’ ‘Page 2’", "Question": "What are the irreversible pulpitis pulp tissue status types?", "Answer": "Acute, Chronic, Necrobiosis."},
"9": {"Source": "‘2-CROWNS-BRIDGES.pdf’ ‘Page 2’", "Question": "Which type of tooth (among root-filled tooth and tooth with vital pulps) is more likely to fracture?", "Answer": "There is a strong clinical impression and some scientific evidence that root-filled teeth are more likely to fracture than teeth with vital pulps."},
"10": {"Source": "‘3-POST AND CORE.pdf’ ‘Page 1’", "Question": "When do we need a post?", "Answer": "When we need to retain the core."},
"11": {"Source": "‘10-MAXILLARY DENTURE FRACTURE.pdf’ ‘Page 2’", "Question": "What are the Methods for Occlusal Equilibration?", "Answer": "a. Swensons Technique: Upper and lower casts are mounted on the articulator. The upper denture is constructed. If the lower natural teeth interfere with the placement of the denture teeth, they are adjusted on the cast, and the area is marked with a pencil. The natural teeth are then modified using the marked diagnostic cast as a guide. This technique is simple but time-consuming.b. Yurkstas Technique: Use of a commercially available U-shaped metal occlusal template that is slightly convex on the lower surface. This template is often an aid in detecting minor deviations in the occlusal scheme.c. Bruce Technique: Use of a clear acrylic resin template fabricated over the modified stone cast. The inner surface of the template is coated with pressure-indicating paste and placed over the patient's natural teeth.d. Boucher's Technique: Use of porcelain teeth on an edentulous cast. Porcelain teeth abrade the opposing dentulous cast. Similar corrections are made on natural dentition."},
"12": {"Source": "‘1-THIRD MOLARS.pdf’ ‘Page 3’", "Question": "What is the third molar decision matrix?", "Answer": "Indication, Surgical Difficulty, Risk of complication, Current health, Future health, Social support, Financial support, Decision."},
"13": {"Source": "‘1-THIRD MOLARS.pdf’ ‘Page 3’", "Question": "If the wound is deep incorporating different tissue layers, what materials should be used in deep closure?", "Answer": "If the wound is deep, thus incorporating different tissue layers, deep closure should be carried out using a resorbable suture material (e.g., polyglycolic acid or chromic gut)."},
"14": {"Source": "‘4- BIOPSY.pdf ‘ ‘Page 12’", "Question": "If the wound is deep incorporating different tissue layers, what materials should be used in deep closure?", "Answer": "If the wound is deep, thus incorporating different tissue layers, deep closure should be carried out using a resorbable suture material (e.g., polyglycolic acid or chromic gut)."}}

'''

In [ ]:
import json
input_data = json.loads(input_json_data)

In [ ]:
def process_data(input_data):
    output_data = {}

    for key, value in input_data.items():
        question = value["Question"]
        answer = prompt(prompt_formatting(question), model, streamer)
        output_data[key] = {"Question": value["Question"], "Source": value["Source"], "Answer": value["Answer"], "LLMAnswer": str(answer)}

    return output_data

In [ ]:
# Process data and prompt for answers
output_data = process_data(input_data)

 Two types of supplementary pins have been recommended in the past to enhance retention: (1) Dual-tipped pins and (2) Grooved pins. The recommended depth for the pin hole into dentine is 1.5-2.0 mm.
 In Australia, the changes in risk profiles due to fluoridation include a 25% reduction in caries incidence, a 27% reduction in caries prevalence, and a 34% reduction in caries-related hospitalizations.
 Cast gold is a good choice for tooth restoration because it is biocompatible, durable, and can be 
customized to the patient's needs. It is also highly corrosion-resistant, which reduces the risk of corrosion-related problems.  
(50 words) 
 Longitudinal fractures are categorized into three:
1.   Craze line: horizontal crack less than 0.1 mm wide.
2.  Chipped tooth: crack >0.1 mm but <1 mm in width.
3.  Fractured tooth: crack >1 mm in width. 

The severity order of these fractures is:  
1.  Craze line: minimal loss of tooth structure (<5% of tooth height).  
2.  Chipped tooth: moderate loss

In [ ]:
output_json_file = "answers.json"
with open(output_json_file, "w") as outfile:
    json.dump(output_data, outfile, indent=2)